In [7]:
import os
import unbsatsubs
DATA_DIR = '../local_data/xml/'
sat_xml = os.path.join(DATA_DIR, 'TestbedSatJun12Flows.xml')

In [10]:
root = unbsatsubs.parsexml_(sat_xml)

In [53]:
i=0
flows = []
for element in root.iter("TestbedSatJun12"):
    flow = {}
    for k in element:
        flow[k.tag] = k.text
    flows.append(flow)    
    
    i = i+1
    if i>5:
        break

In [54]:
flows

[{'Tag': 'Normal',
  'appName': 'HTTPWeb',
  'destination': '216.246.64.49',
  'destinationPayloadAsBase64': None,
  'destinationPayloadAsUTF': None,
  'destinationPort': '80',
  'destinationTCPFlagsDescription': 'F,A',
  'direction': 'L2R',
  'protocolName': 'tcp_ip',
  'source': '192.168.1.104',
  'sourcePayloadAsBase64': None,
  'sourcePayloadAsUTF': None,
  'sourcePort': '22441',
  'sourceTCPFlagsDescription': 'F,A',
  'startDateTime': '2010-06-12T23:58:53',
  'stopDateTime': '2010-06-12T23:58:53',
  'totalDestinationBytes': '64',
  'totalDestinationPackets': '1',
  'totalSourceBytes': '128',
  'totalSourcePackets': '2'},
 {'Tag': 'Normal',
  'appName': 'HTTPWeb',
  'destination': '216.246.64.66',
  'destinationPayloadAsBase64': None,
  'destinationPayloadAsUTF': None,
  'destinationPort': '80',
  'destinationTCPFlagsDescription': 'F,A',
  'direction': 'L2R',
  'protocolName': 'tcp_ip',
  'source': '192.168.1.104',
  'sourcePayloadAsBase64': None,
  'sourcePayloadAsUTF': None,
  's